<a href="https://colab.research.google.com/github/HoonC-corgi/Machine_Learning_SelfStudy/blob/main/Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest Algorythm

## Random Forest Classifier

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [9]:
# cross_validate() 메소드를 통한 교차검증 수행
# return_train_score 매개변수를 True로 지정함으로써 훈련 세트에 대한 점수도 동시에 반환
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [12]:
# 랜덤 포레스트는 결정 트리의 앙상블이기에 특성 중요도를 계산할수 있다
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 랜덤 포레스트는 특정 특성에 과도하게 집중하지 않고 다양한 특성이 훈련에 기여할 기회를 얻기에 과대적합의 완화와 일반화 성능을 높임

[0.23167441 0.50039841 0.26792718]


In [13]:
# 랜덤 포레스트 분류는 자체적으로 모델 평가 점수를 얻을 수 있음.
# 부트스트랩 샘플에 포함되지 않고 남게 되는 OOB샘플을 통해 훈련된 결정트리를 평가
# oob_score 매개변수를 True로 지정하고 OOB 점수의 평균을 구함
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리
      엑스트라 트리는 부트스트랩 샘플을 사용하지 않고, 무작위로 샘플링함 결정트리의 splitter='random'과 같음

In [16]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 일반적으로 엑스트라 트리가 무작위성이 크기에 더 많은 결정트리를 훈련해야 하지만, 랜덤하게 노드를 분할하기에 계산 속도가 빠르다

0.9974503966084433 0.8887848893166506


In [17]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# Gradient Boosting
      얕은 깊이의 결정트리를 사용하여 이전트리의 오차를 보완하는 앙상블 방식

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 강함

0.8881086892152563 0.8720430147331015


In [24]:
# 학습률과 결정트리의 개수를 증가시킴으로 성능을 향상시킬 수 있음
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [26]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 그레이디언트 부스팅은 랜덤 포레스트보다 일부 특성에 더 집중함

[0.15872278 0.68010884 0.16116839]


# Histogrm-based Gradient Boosting
       입력 특성을 256개의 구간으로 우선적으로 나누고 노드를 분할 하므로 최적의 분할을 빠르게 찾을 수 있음, 누락된 특성을 전처리할 필요 없음
       max_iter 매개변수를 통해 성능을 향상 시킬 수 있음

In [29]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [30]:
# 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산을 위해서는 permutation_importance() 메소드를 사용함, 특성을 하나씩 랜덤 셔플하여 모델 성능의 변화를 관찰하여 중요도를 계산
# n_repeats 매개변수는 랜덤 셔플 횟수를 지정
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [31]:
# 히스토그램 기반 그레이디언트 부스팅 분류의 최종 성능을 구하면
hgb.score(test_input, test_target)

0.8723076923076923

## XGBoost를 이용한 그레이디언트 부스팅 구현

In [34]:
# 그레이디언트 부스팅 알고리즘을 지원하는 대표 라이브러리 XGBoost를 통해 이를 구현
# tree_method 매개변수를 'hist'로 지정
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


## LightGBM을 이용한 그레이디언트 부스팅 구현

In [36]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


# 전체 소스코드

# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

In [ ]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 총정리
  ## 키워드
  ### 앙상블 학습
          정형 데이터를 훈련하여 각 모델의 예측을 취합하여 최종 결과를 만드는 학습 방식

  ### 랜덤 포레스트
          대표적인 결정 트리 기반의 앙상블 학습 방법, 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만듦

  ### 엑스트라 트리
          랜덤포레스트와 비슷하지만 부트스트랩 샘플을 사용하지 않고 랜덤하게 노드를 분할해 과대적합을 감소시킴

  ### 그레이디언트 부스팅
          랜덤 포레스트, 엑스트라 트리와 다르게 결정 트리를 연속적으로 추가하여 손실 함수를 최소화 하는 앙상블 방법, 훈련 속도가 느리고 높은 성능을 보임
          이를 개선한 히스토그램 기반 그레이디언트 부스팅이 안정적이고 높은 성능을 보임


  ## 핵심 패키지와 함수
  ## scikit-learn
  ### RandomForestClassifier
          n_estimators 매개변수를 통해 트리의 개수를 지정, 디폴트는 100
          criterion 매개변수를 통해 불순도를 지정, 디폴트는 지니 불순도 'gini'와 엔트로피 불순도 'entropy'가 있음
          max_depth를 통해 트리의 최대 성장 깊이를 지정, 디폴트는 None으로 리프 노드가 순수하거나 min_samples_split보다 샘플 개수가 적을 때까지 성장
          min_samples_split은 노드를 나누기 위한 최소 샘플 개수, 디폴트는 2
          max_features 매개변수는 최적의 분할을 위해 탐색할 특성의 개수를 지정, 디폴트는 auto로 특성 개수의 제곱근
          bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정, 디폴트는 True
          oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정, 디폴트는 False
          n_jobs를 통해 병렬 실행에 사용할 CPU 코어수를 지정, 디폴트는 -1으로 보든 코어를 사용

  ### ExtraTreesClassifier
          bootstrap 매개변수를 통해 부트스트랩 샘플의 사용여부를 지정, 디폴트는 False
          oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정, 디폴트는 False
          n_jobs를 통해 병렬 실행에 사용할 CPU 코어수를 지정, 디폴트는 -1으로 보든 코어를 사용

  ### GradientBoostingClassifier
        loss 매개변수는 손실함수를 지정, 디폴트는 로지스틱 손실 함수인 'deviance'
        learning_rate 매개변수는 트리가 앙상블에 기여하는 정도를 조절, 디폴트는 0.1
        n_estimators 매개변수를 통해 부스팅 단계를 수행하는 트리의 개수를 지정, 디폴트는 100
        subsample 매개변수를 통해 사용할 훈련 세트의 샘플 비율을 지정, 디폴트는 1.0
        max_depth 매개변수를 통해 개별 회귀 트리의 최대 깊이를 지정, 디폴트는 3

  ### HistGradientBoostingClassifier
        learning_rate 매개변수는 학습률 또는 감쇠율이라 함, 디폴트는 0.1이며 1.0이면 감쇠가 전혀 없음
        max_iter는 부스팅 단계를 수행하는 트리의 개수, 디폴트는 100
        max_bins는 입력 데이터를 나눌 구간의 개수로, 디폴트는 255이며 이보다 클 수 없음


  # 앙상블 학습 정리
      앙상블 학습은 정형 데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘

  ## 사이킷런

  ### 랜덤 포레스트
        부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘
  ### 엑스트라 트리
        결정 트리의 노드를 랜덤하게 분할
  ### 그레이디언트 부스팅
        이전 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가
  ### 히스토그램 기반 그레이디언트 부스팅
        훈련 데이터를 256개의 정수 구간으로 나누어 빠르고 높은 성능을 냄, 255개를 디폴트로 사용, 나머지 하나는 누락된 클래스를 위해 사용 됨


  ## 그외 라이브러리
  ### XGBoost
  ### LightGBM